In [27]:
import math

# Função para ler o arquivo de entrada e construir o estado inicial e o objetivo
def ler_configuracao(arquivo):
    with open(arquivo, 'r') as f:
        linhas = f.readlines()
    processadas = []
    for linha in linhas:
        linha = linha.strip()
        if not linha:
            continue
        if linha.startswith('#'):
            continue
        linha = linha.split('#')[0].strip()
        if linha:
            processadas.append(linha)
    num_posicoes = int(processadas[0])
    max_pilha = int(processadas[1])
    estado_inicial = []
    boxes = []
    for i in range(num_posicoes):
        linha = processadas[2 + i]
        if linha.strip() == "B":
            estado_inicial.append("B")
        else:
            valores = [int(x) for x in linha.split() if int(x) != 0]
            estado_inicial.append(tuple(valores))
            boxes.extend(valores)
    estado_inicial = tuple(estado_inicial)
    total_boxes = len(boxes)
    disponiveis = [i for i, pos in enumerate(estado_inicial) if pos != "B"]
    sorted_boxes = sorted(boxes, reverse=True)
    num_stacks = math.ceil(len(sorted_boxes) / max_pilha)
    objetivo = list(estado_inicial)
    box_index = 0
    for idx in disponiveis:
        if num_stacks > 0:
            stack = tuple(sorted_boxes[box_index: box_index + max_pilha])
            objetivo[idx] = stack
            box_index += max_pilha
            num_stacks -= 1
        else:
            objetivo[idx] = ()
    estado_objetivo = tuple(objetivo)
    return num_posicoes, max_pilha, estado_inicial, estado_objetivo, total_boxes

In [28]:
from queue import PriorityQueue

class No:
    def __init__(self, estado, no_pai=None, aresta=None, custo=0.0, heuristica=0.0):
        self.estado = estado            # Estado é uma tupla de tuplas (cada pilha é imutável)
        self.no_pai = no_pai            
        self.aresta = aresta            # Ação realizada para chegar neste nó (ex: (origem, destino, caixa))
        self.custo = custo              
        self.heuristica = heuristica    

    def __repr__(self):
        return str(self.estado)

    def __lt__(self, outro):
        # Critério de comparação: custo acumulado + heurística
        return (self.custo + self.heuristica) < (outro.custo + outro.heuristica)


In [29]:
# Função para recuperar a sequência de estados do nó inicial ao nó final
def no_caminho(no):
    caminho = [no.estado]
    while no.no_pai is not None:
        caminho.append(no.estado)
        no = no.no_pai
    caminho.reverse()
    return caminho

In [30]:
# Função para recuperar a sequência de ações (movimentos realizados)
def vertice_caminho(no):
    caminho = []
    while no.no_pai is not None:
        if no.aresta is not None:
            caminho.append(no.aresta)
        no = no.no_pai
    caminho.reverse()
    return caminho

In [31]:
class Visitados:
    def __init__(self):
        self.visitados = set()

    def adicionar(self, no):
        self.visitados.add(no.estado)

    def foi_visitado(self, no):
        return no.estado in self.visitados

    def tamanho(self):
        return len(self.visitados)

In [32]:
class FilaPrioridade:
    def __init__(self):
        self.fila = PriorityQueue()

    def push(self, valor, item):
        self.fila.put((valor, item))

    def pop(self):
        if self.esta_vazio():
            return None
        else:
            (_, no) = self.fila.get()
            return no

    def esta_vazio(self):
        return self.fila.empty()

In [33]:
class Problema:
    def iniciar(self):
        raise NotImplementedError

    def imprimir(self, no):
        raise NotImplementedError

    def testar_objetivo(self, no):
        raise NotImplementedError

    def gerar_sucessores(self, no):
        raise NotImplementedError

    def custo(self, no, no_sucessor):
        raise NotImplementedError

    def heuristica(self, estado):
        raise NotImplementedError

In [34]:
# Classe para empilhamento de caixas
class EmpilhaCaixas(Problema):
    def __init__(self, arquivo):
        self.num_posicoes, self.limite_pilha, self.estado_inicial, self.estado_objetivo, self.total_caixas = ler_configuracao(arquivo)

    def iniciar(self):
        no_raiz = No(self.estado_inicial, custo=0.0, heuristica=self.heuristica(self.estado_inicial))
        return no_raiz

    # Imprime o estado final no formato esperado, preenchendo com 0 para completar as pilhas
    def imprimir(self, no):
        s = "# configuração final das {} posições\n".format(self.num_posicoes)
        for pos in no.estado:
            if pos == "B":
                s += "B\n"
            else:
                pilha = list(pos)
                if len(pilha) < self.limite_pilha:
                    pilha.extend([0] * (self.limite_pilha - len(pilha)))
                s += " ".join(map(str, pilha)) + "\n"
        return s

    def testar_objetivo(self, no):
        return no.estado == self.estado_objetivo

    def gerar_sucessores(self, no):
        sucessores = []
        estado_atual = no.estado
        for i in range(self.num_posicoes):
            if estado_atual[i] == "B" or len(estado_atual[i]) == 0:
                continue
            caixa = estado_atual[i][-1]
            for j in range(self.num_posicoes):
                if i == j or estado_atual[j] == "B":
                    continue
                if len(estado_atual[j]) >= self.limite_pilha:
                    continue
                if len(estado_atual[j]) > 0 and estado_atual[j][-1] <= caixa:
                    continue
                novo_estado = []
                for k in range(self.num_posicoes):
                    if estado_atual[k] == "B":
                        novo_estado.append("B")
                    else:
                        novo_estado.append(list(estado_atual[k]))
                novo_estado[i].pop()
                novo_estado[j].append(caixa)
                for k in range(self.num_posicoes):
                    if novo_estado[k] != "B":
                        novo_estado[k] = tuple(novo_estado[k])
                novo_estado = tuple(novo_estado)
                acao = (i, j, caixa)
                custo_mov = self.calcular_custo(i, j, caixa)
                novo_no = No(novo_estado, no, acao, no.custo + custo_mov, self.heuristica(novo_estado))
                sucessores.append(novo_no)
        return sucessores

    def calcular_custo(self, origem, destino, caixa):
        distancia = abs(origem - destino)
        if distancia == 1:
            # Modo local: move 1 casa com custo 1 de energia
            custo_mov = 1
        else:
            # Modo de extensão total: independente da distância, custo fixo de 3 de energia (equivalente a 4 casas)
            custo_mov = 3
        custo_peso = caixa / 10.0  # Acréscimo de custo por peso
        return custo_mov + custo_peso


    def custo(self, no, no_sucessor):
        return no_sucessor.custo - no.custo

    def heuristica(self, estado):
        # Cria um mapeamento: para cada caixa (peso) do estado objetivo (em posições não "B")
        # associa a coluna (posição) em que ela deve estar.
        mapping = {}
        for j in range(self.num_posicoes):
            if self.estado_objetivo[j] == "B":
                continue
            for box in self.estado_objetivo[j]:
                mapping[box] = j

        h = 0.0
        # Para cada posição (exceto o braço) do estado atual:
        for i in range(self.num_posicoes):
            if estado[i] == "B":
                continue
            # Para cada caixa na pilha atual:
            for box in estado[i]:
                # Obtém a coluna alvo para esta caixa (se não houver, assume que já está no lugar)
                target_col = mapping.get(box, i)
                distance = abs(i - target_col)
                # Custo de movimentação: se 1 casa, custo 1; se >1, custo = distância * 0.75
                if distance == 0:
                    move_cost = 0
                elif distance == 1:
                    move_cost = 1
                else:
                    move_cost = distance * 0.75
                # Custo adicional pelo peso (cada 10kg custa 1)
                weight_cost = box / 10.0
                h += move_cost + weight_cost
        return h

In [35]:
# Função para formatar os movimentos do braço mecânico
def formatar_movimentos(acoes, estado_inicial):
    pos_braco = None
    for i, pos in enumerate(estado_inicial):
        if pos == "B":
            pos_braco = i
            break
    movimentos = []
    for acao in acoes:
        origem, destino, caixa = acao
        d1 = abs(pos_braco - origem)
        direcao1 = "E" if origem < pos_braco else "D"
        custo_p = d1 * 2
        movimentos.append(f"{direcao1} {d1} P {caixa} {custo_p}")
        pos_braco = origem
        d2 = abs(pos_braco - destino)
        direcao2 = "E" if destino < pos_braco else "D"
        custo_s = d2 * 5
        movimentos.append(f"{direcao2} {d2} S {caixa} {custo_s}")
        pos_braco = destino
    return "\n".join(movimentos)

In [36]:
# Dijkstra (A* com h = 0)
def dijkstra(problema):
    no_inicial = problema.iniciar()
    fila = FilaPrioridade()
    # Usa somente o custo acumulado como chave na fila
    fila.push(no_inicial.custo, no_inicial)
    visitados = Visitados()
    visitados.adicionar(no_inicial)

    while not fila.esta_vazio():
        no_atual = fila.pop()
        if problema.testar_objetivo(no_atual):
            return (visitados.tamanho(), no_atual)
        for sucessor in problema.gerar_sucessores(no_atual):
            if not visitados.foi_visitado(sucessor):
                visitados.adicionar(sucessor)
                fila.push(sucessor.custo, sucessor)
    return (visitados.tamanho(), None)

In [37]:
# A* (utiliza custo acumulado + heurística)
def a_estrela(problema):
    no_inicial = problema.iniciar()
    fila = FilaPrioridade()
    fila.push(no_inicial.custo + no_inicial.heuristica, no_inicial)
    visitados = Visitados()
    visitados.adicionar(no_inicial)

    while not fila.esta_vazio():
        no_atual = fila.pop()
        if problema.testar_objetivo(no_atual):
            return (visitados.tamanho(), no_atual)
        for sucessor in problema.gerar_sucessores(no_atual):
            if not visitados.foi_visitado(sucessor):
                visitados.adicionar(sucessor)
                fila.push(sucessor.custo + sucessor.heuristica, sucessor)
    return (visitados.tamanho(), None)


In [38]:
# ganancioso (Greedy) – prioriza somente a heurística
def greedy(problema):
    no_inicial = problema.iniciar()
    fila = FilaPrioridade()
    fila.push(no_inicial.heuristica, no_inicial)
    visitados = Visitados()
    visitados.adicionar(no_inicial)

    while not fila.esta_vazio():
        no_atual = fila.pop()
        if problema.testar_objetivo(no_atual):
            return (visitados.tamanho(), no_atual)
        for sucessor in problema.gerar_sucessores(no_atual):
            if not visitados.foi_visitado(sucessor):
                visitados.adicionar(sucessor)
                fila.push(sucessor.heuristica, sucessor)
    return (visitados.tamanho(), None)


In [39]:
# executa os três algoritmos e mostra seus resultados
if __name__ == "__main__":
    problema = EmpilhaCaixas("input.txt")
    with open("output.txt", "w") as f:
        # Busca A*
        f.write("=== Busca A* ===\n")
        qtd, no_sol = a_estrela(problema)
        if no_sol is None:
            f.write("Não houve solução para o problema.\n")
        else:
            f.write(problema.imprimir(no_sol))
            f.write("\n# movimentos do braço mecânico até o estado final\n")
            acoes = vertice_caminho(no_sol)
            movimentos_str = formatar_movimentos(acoes, problema.estado_inicial)
            f.write(movimentos_str)
        f.write("\nEstados visitados: " + str(qtd) + "\n\n")
        
        # Busca Gananciosa (Greedy)
        f.write("=== Busca Gananciosa (Greedy) ===\n")
        qtd, no_sol = greedy(problema)
        if no_sol is None:
            f.write("Não houve solução para o problema.\n")
        else:
            f.write(problema.imprimir(no_sol))
            f.write("\n# movimentos do braço mecânico até o estado final\n")
            acoes = vertice_caminho(no_sol)
            movimentos_str = formatar_movimentos(acoes, problema.estado_inicial)
            f.write(movimentos_str)
        f.write("\nEstados visitados: " + str(qtd) + "\n\n")
        
        # Busca Dijkstra
        f.write("=== Busca Dijkstra ===\n")
        qtd, no_sol = dijkstra(problema)
        if no_sol is None:
            f.write("Não houve solução para o problema.\n")
        else:
            f.write(problema.imprimir(no_sol))
            f.write("\n# movimentos do braço mecânico até o estado final\n")
            acoes = vertice_caminho(no_sol)
            movimentos_str = formatar_movimentos(acoes, problema.estado_inicial)
            f.write(movimentos_str)
        f.write("\nEstados visitados: " + str(qtd) + "\n")
    print("Saída gerada no arquivo 'output.txt'.")

Saída gerada no arquivo 'output.txt'.
